In [5]:
import os
import shutil
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [6]:
os.getcwd()

'C:\\Users\\SABYA\\Aritar01\\CAP_106\\Raw_Data'

In [7]:
path = "C:/Users/SABYA/Aritar01/CAP_106/Raw_Data"
# change the current working dir to path
os.chdir(path)
os.getcwd()

'C:\\Users\\SABYA\\Aritar01\\CAP_106\\Raw_Data'

In [4]:
# create an empty dataframe
df = pd.DataFrame()
for run_no in range(1,16):
    # read the dcs data
    df1 = pd.read_excel("BA_106_Seperate_Active_Run_Data.xlsx", sheet_name=f"RUN {run_no}")
     
    
    # read the additional data
    df2 = pd.read_excel("Additional_BA_106_Runs_seperated.xlsx", sheet_name=f"RUN {run_no}")
    # store the date column from additional data
    date2 = df2['Date']
    # access the first date stamp from date2
    dt2 = date2[0]
    # remove the minute, sec and microsecond from dt1 date stamp
    new_dt2 = dt2.replace(minute=0, second=0, microsecond=0) + datetime.timedelta(hours=1)
    # create a date range
    ls2 = list(pd.date_range(start=new_dt2, periods=df2.shape[0], freq='1H'))
    # drop old date column from df2
    df2 = df2.drop(labels=['Date'], axis=1)
    # insert newly created date list to df1
    df2.insert(0, 'Date', ls2)
    
    
    # columns to keep from df1
    ls_of_cols = ['Date',
                 'RUN Duration, hours',
                 'RUN No.',
                 'NpahthaFeedCoil1',
                 'NpahthaFeedCoil2',
                 'NpahthaFeedCoil3',
                 'NpahthaFeedCoil4',
                 'NpahthaFeedCoil5',
                 'NpahthaFeedCoil6',
                 'DS RateCoil1',
                 'DS RateCoil2',
                 'DS RateCoil3',
                 'DS RateCoil4',
                 'DS RateCoil5',
                 'DS RateCoil6',
                 'COT1',
                 'COT2',
                 'COT3',
                 'COT4',
                 'COT5',
                 'COT6',
                 'CIP1',
                 'CIP2',
                 'CIP3',
                 'CIP4',
                 'CIP5',
                 'CIP6',
                 'Wall_Fuel_Gas_Flow',
                 'Ground_Fuel_Gas_Flow',
                 'O2',
                 'CrossOverTemp_Coil1',
                 'CrossOverTemp_Coil2',
                 'CrossOverTemp_Coil3',
                 'CrossOverTemp_Coil4',
                 'CrossOverTemp_Coil5',
                 'CrossOverTemp_Coil6']
    # store this columns into df3
    df3 = df1[ls_of_cols]
    # calulate mean, max and average of individual colis
    total_naphtha_feed = df3[['NpahthaFeedCoil1',
     'NpahthaFeedCoil2',
     'NpahthaFeedCoil3',
     'NpahthaFeedCoil4',
     'NpahthaFeedCoil5',
     'NpahthaFeedCoil6']].sum(axis=1)
    total_ds_rate = df3[['DS RateCoil1',
     'DS RateCoil2',
     'DS RateCoil3',
     'DS RateCoil4',
     'DS RateCoil5',
     'DS RateCoil6']].sum(axis=1)
    avg_cot = df3[['COT1',
     'COT2',
     'COT3',
     'COT4',
     'COT5',
     'COT6']].mean(axis=1)
    max_cip = df3[['CIP1',
     'CIP2',
     'CIP3',
     'CIP4',
     'CIP5',
     'CIP6'
     ]].max(axis=1)
    avg_cross_over_temp = df3[['CrossOverTemp_Coil1',
     'CrossOverTemp_Coil2',
     'CrossOverTemp_Coil3',
     'CrossOverTemp_Coil4',
     'CrossOverTemp_Coil5',
     'CrossOverTemp_Coil6']].mean(axis=1)

    ds_ratio_coil1 = df3["DS RateCoil1"] / df3["NpahthaFeedCoil1"]
    ds_ratio_coil2 = df3["DS RateCoil2"] / df3["NpahthaFeedCoil2"]
    ds_ratio_coil3 = df3["DS RateCoil3"] / df3["NpahthaFeedCoil3"]
    ds_ratio_coil4 = df3["DS RateCoil4"] / df3["NpahthaFeedCoil4"]
    ds_ratio_coil5 = df3["DS RateCoil5"] / df3["NpahthaFeedCoil5"]
    ds_ratio_coil6 = df3["DS RateCoil6"] / df3["NpahthaFeedCoil6"]
    total_ds_ratio = total_ds_rate / total_naphtha_feed
    
    # add the above newly created columns to df3
    df3["DSRatioCoil1"] = ds_ratio_coil1
    df3["DSRatioCoil2"] = ds_ratio_coil2
    df3["DSRatioCoil3"] = ds_ratio_coil3
    df3["DSRatioCoil4"] = ds_ratio_coil4
    df3["DSRatioCoil5"] = ds_ratio_coil5
    df3["DSRatioCoil6"] = ds_ratio_coil6
    df3["Total_Naphtha_Feed"] = total_naphtha_feed
    df3["Total_DS_Rate"] = total_ds_rate
    df3["Total_Ds_Ratio"] = total_ds_ratio
    df3["Avg_COT"] = avg_cot
    df3["Avg_CIT"] = avg_cross_over_temp
    df3["Max_CIP"] = max_cip
    
    # keep all columns but date from additional dataframe df2
    df4 = df2[df2.columns[1:].tolist()]
    
    # concat df3 and df4 into df5
    df5 = pd.concat([df3, df4], axis=1)
    
    # append df5 into pre-assigned empty dataframe
    df = df.append(df5)
    
    
df.to_excel("BA_106_final_data.xlsx", index=False)

In [16]:
# read the final data
df_final = pd.read_excel("BA_106_final_data.xlsx")

In [17]:
df_final

,Date,"RUN Duration, hours",RUN No.,NpahthaFeedCoil1,NpahthaFeedCoil2,NpahthaFeedCoil3,NpahthaFeedCoil4,NpahthaFeedCoil5,NpahthaFeedCoil6,DS RateCoil1,...,Total_Naphtha_Feed,Total_DS_Rate,Total_Ds_Ratio,Avg_COT,Avg_CIT,Max_CIP,BFW Rate,DS Temp,Feed Temp1,Feed Temp2
0,2021-01-07 01:00:00,1.3,1,5.252560,5.252560,5.343612,5.064888,4.829841,5.064751,3.302164,...,30.808213,19.814357,0.643152,820.259940,636.873827,1.075892,1.178144e-16,251.296965,50.650454,59.879870
1,2021-01-07 02:00:00,1.4,1,5.277761,5.277761,5.337419,5.057818,4.826202,5.067631,3.301951,...,30.844594,19.809631,0.642240,820.040961,634.427615,1.055169,1.178144e-16,253.302595,51.009955,60.187294
2,2021-01-07 03:00:00,1.5,1,5.260479,5.260479,5.343015,5.052432,4.847977,5.058545,3.302150,...,30.822927,19.812698,0.642791,820.059912,634.207583,1.060699,1.178144e-16,254.730001,51.108173,60.164672
3,2021-01-07 04:00:00,1.6,1,5.294079,5.294079,5.348264,5.052154,4.863041,5.008254,3.301754,...,30.859871,19.811996,0.641999,819.834424,635.087561,1.063285,1.178144e-16,255.202556,51.248020,60.059837
4,2021-01-07 05:00:00,1.7,1,5.340085,5.340085,5.431115,5.139928,4.892867,5.091473,3.343947,...,31.235552,20.063034,0.642314,819.353981,635.992299,1.078325,1.178144e-16,253.684494,51.260781,59.829362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11300,2022-05-31 19:00:00,59.5,15,5.709997,5.709997,5.814525,5.844336,5.951330,5.891736,3.489884,...,34.921922,20.939064,0.599597,817.960936,639.801104,1.469659,4.271138e+01,233.667061,50.017260,53.136514
11301,2022-05-31 20:00:00,59.6,15,5.709468,5.709468,5.810093,5.839333,5.952943,5.908858,3.490102,...,34.930163,20.940725,0.599503,818.299786,640.889692,1.472635,4.265844e+01,233.670287,49.536603,52.846304
11302,2022-05-31 21:00:00,59.7,15,5.759999,5.759999,5.866227,5.892238,5.984259,5.907374,3.489808,...,35.170096,20.938667,0.595354,817.913417,638.380010,1.469588,4.261256e+01,233.371803,49.418948,53.049036
11303,2022-05-31 22:00:00,59.8,15,5.802558,5.802558,5.914639,5.941456,6.035958,5.932698,3.490195,...,35.429867,20.940745,0.591048,818.189503,637.892354,1.472233,4.270312e+01,233.243962,50.064544,52.951541


In [18]:
# separate the date column
date = df_final['Date']
df_final = df_final.drop(labels=['Date'], axis=1)

In [19]:
# datatype conversion from object to float
df_final = df_final.astype('float')

In [ ]:
df_final["Avg_Feed_Temp"] = df_final[["Feed Temp1", "Feed Temp2"]].mean(axis=1)

In [ ]:
# paths to store the correlation heatmaps and correlation dataframe
heat_path = "C:/Users/SABYA/Aritar01/CAP_106/CAP_106_Research_Env/EDA_Report/Heatmaps"
data_path = "C:/Users/SABYA/Aritar01/CAP_106/CAP_106_Research_Env/EDA_Report/CorrInfo"

In [ ]:
# correlation analysis for Naphtha Feed
df_naphtha_feed = df_final[['NpahthaFeedCoil1',
                        'NpahthaFeedCoil2', 
                        'NpahthaFeedCoil3',
                        'NpahthaFeedCoil4',
                        'NpahthaFeedCoil5', 
                        'NpahthaFeedCoil6',
                        'Total_Naphtha_Feed']]

In [ ]:
# computing the upper triangle correlation matrix
corr_mat_1 = df_naphtha_feed.corr().abs()
upper_tri_1 = corr_mat_1.where(np.triu(np.ones(corr_mat_1.shape), k=1).astype(np.bool))
upper_tri_1

In [ ]:
# store the corr dataframe for Naphtha Feed
df_naptha_corr = pd.DataFrame(corr_mat_1["Total_Naphtha_Feed"])
df_naptha_corr.to_excel(data_path + "/naphtha_feed_corr_info.xlsx")

In [ ]:
# plotting correlation heatmap for Naphtha Feed
dataplot = sb.heatmap(upper_tri_1, cmap="Blues", annot=True)
  
# displaying heatmap
# plt.show()

# save the plot as .png
plt.savefig(heat_path +"/naphtha_feed_corr_heatmap.png")

In [ ]:
# finding index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper_tri_1.columns if any(upper_tri_1[column] > 0.95)]
print(); print(to_drop) 

In [ ]:
# correlation analysis for COT
df_cot = df_final[['COT1',
                   'COT2', 
                   'COT3', 
                   'COT4', 
                   'COT5', 
                   'COT6', 
                   'Avg_COT']]

In [ ]:
# computing the upper triangle correlation matrix
corr_mat_2 = df_cot.corr().abs()
upper_tri_2 = corr_mat_2.where(np.triu(np.ones(corr_mat_2.shape), k=1).astype(np.bool))
upper_tri_2

In [ ]:
# store the corr dataframe for COT
df_cot_corr = pd.DataFrame(corr_mat_2["Avg_COT"])
df_cot_corr.to_excel(data_path + "/cot_corr_info.xlsx")

In [ ]:
# plotting correlation heatmap for COT
dataplot = sb.heatmap(upper_tri_2, cmap="Blues", annot=True)
  
# displaying heatmap
# plt.show()

# save the plot as .png
plt.savefig(heat_path +"/cot_corr_heatmap.png")

In [ ]:
# correlation analysis for DS Ratio
df_ds_ratio = df_final[['DSRatioCoil1',
                    'DSRatioCoil2',
                    'DSRatioCoil3',
                    'DSRatioCoil4',
                    'DSRatioCoil5',
                    'DSRatioCoil6',
                    'Total_Ds_Ratio']]

In [ ]:
# computing the upper triangle correlation matrix
corr_mat_3 = df_ds_ratio.corr().abs()
upper_tri_3 = corr_mat_3.where(np.triu(np.ones(corr_mat_3.shape), k=1).astype(np.bool))
upper_tri_3

In [ ]:
# store the corr dataframe for DS Ratio
df_ds_ratio_corr = pd.DataFrame(corr_mat_3["Total_Ds_Ratio"])
df_ds_ratio_corr.to_excel(data_path + "/ds_ratio_corr_info.xlsx")

In [ ]:
# plotting correlation heatmap for COT
dataplot = sb.heatmap(upper_tri_3, cmap="Blues", annot=True)
  
# displaying heatmap
# plt.show()

# save the plot as .png
plt.savefig(heat_path +"/ds_ratio_corr_heatmap.png")

In [ ]:
# correlation analysis for CrossOver Temperature
df_cit = df_final[['CrossOverTemp_Coil1',
                        'CrossOverTemp_Coil2',
                        'CrossOverTemp_Coil3',
                        'CrossOverTemp_Coil4',
                        'CrossOverTemp_Coil5',
                        'CrossOverTemp_Coil6',
                        'Avg_CIT']]

In [ ]:
# computing the upper triangle correlation matrix
corr_mat_4 = df_cit.corr().abs()
upper_tri_4 = corr_mat_4.where(np.triu(np.ones(corr_mat_4.shape), k=1).astype(np.bool))
upper_tri_4

In [ ]:
# store the corr dataframe for CIT
df_cit_corr = pd.DataFrame(corr_mat_4["Avg_CIT"])
df_cit_corr.to_excel(data_path + "/cit_corr_info.xlsx")

In [ ]:
# plotting correlation heatmap for CIT
dataplot = sb.heatmap(upper_tri_4, cmap="Blues", annot=True)
  
# displaying heatmap
# plt.show()

# save the plot as .png
plt.savefig(heat_path +"/cit_corr_heatmap.png")

In [ ]:
# finding index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper_tri_4.columns if any(upper_tri_4[column] > 0.95)]
print(); print(to_drop) 

In [ ]:
# correlation analysis for Coil Inlet Pressure
df_cip = df_final[['CIP1',
                    'CIP2',
                    'CIP3',
                    'CIP4',
                    'CIP5',
                    'CIP6',
                    'Max_CIP']]

In [ ]:
# computing the upper triangle correlation matrix
corr_mat_5 = df_cip.corr().abs()
upper_tri_5 = corr_mat_5.where(np.triu(np.ones(corr_mat_5.shape), k=1).astype(np.bool))
upper_tri_5

In [ ]:
# store the corr dataframe for CIP
df_cip_corr = pd.DataFrame(corr_mat_5["Max_CIP"])
df_cip_corr.to_excel(data_path + "/cip_corr_info.xlsx")

In [ ]:
# plotting correlation heatmap for CIT
dataplot = sb.heatmap(upper_tri_5, cmap="Blues", annot=True)
  
# displaying heatmap
# plt.show()

# save the plot as .png
plt.savefig(heat_path +"/cip_corr_heatmap.png")

In [ ]:
# finding index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper_tri_5.columns if any(upper_tri_5[column] > 0.95)]
print(); print(to_drop) 

In [ ]:
# correlation analysis for Feed Temperature
df_feed_temp = df_final[['Feed Temp1',
                        'Feed Temp2']]

In [ ]:
# computing the upper triangle correlation matrix
corr_mat_6 = df_feed_temp.corr().abs()
upper_tri_6 = corr_mat_6.where(np.triu(np.ones(corr_mat_6.shape), k=1).astype(np.bool))
upper_tri_6

In [ ]:
# store the corr dataframe for Feed Temperature
df_feed_temp_corr = pd.DataFrame(corr_mat_6["Feed Temp1"])
df_feed_temp_corr.to_excel(data_path + "/feed_temp_corr_info.xlsx")

In [ ]:
# plotting correlation heatmap for Feed Temperature
dataplot = sb.heatmap(upper_tri_6, cmap="Blues", annot=True)
  
# displaying heatmap
# plt.show()

# save the plot as .png
plt.savefig(heat_path +"/feed_temp_corr_heatmap.png")

In [20]:
# correlation analysis in between 
#  'Total_Naphtha_Feed', 
#  'Avg_COT', 
#  'Total_Ds_Ratio', 
#  'Wall_Fuel_Gas_Flow', 
#  'Ground_Fuel_Gas_Flow',
#  'O2', 
#  'CrossOverTemp_Coil1',
#  'CrossOverTemp_Coil2',
#  'CrossOverTemp_Coil3',
#  'CrossOverTemp_Coil4',
#  'CrossOverTemp_Coil5',
#  'CrossOverTemp_Coil6',
#  'CIP1',
#  'CIP2',
#  'CIP3',
#  'CIP4',
#  'CIP5',
#  'CIP6',
#  'BFW Rate',
#  'DS Temp',
#  'Feed Temp1',
#  'Feed Temp2'
df_other_features = df_final[[  'Total_Naphtha_Feed', 
 'Avg_COT', 
 'Total_Ds_Ratio', 
 'Wall_Fuel_Gas_Flow', 
 'Ground_Fuel_Gas_Flow',
 'O2', 
 'CrossOverTemp_Coil1',
 'CrossOverTemp_Coil2',
 'CrossOverTemp_Coil3',
 'CrossOverTemp_Coil4',
 'CrossOverTemp_Coil5',
 'CrossOverTemp_Coil6',
 'CIP1',
 'CIP2',
 'CIP3',
 'CIP4',
 'CIP5',
 'CIP6',
 'BFW Rate',
 'DS Temp',
 'Feed Temp1',
 'Feed Temp2'
]]

In [21]:
df_other_features = pd.concat([date, df_other_features], axis=1)

In [22]:
df_other_features

,Date,Total_Naphtha_Feed,Avg_COT,Total_Ds_Ratio,Wall_Fuel_Gas_Flow,Ground_Fuel_Gas_Flow,O2,CrossOverTemp_Coil1,CrossOverTemp_Coil2,CrossOverTemp_Coil3,...,CIP1,CIP2,CIP3,CIP4,CIP5,CIP6,BFW Rate,DS Temp,Feed Temp1,Feed Temp2
0,2021-01-07 01:00:00,30.808213,820.259940,0.643152,0.530118,4.102766,2.353679,630.542551,633.943171,632.920609,...,1.024428,0.962958,1.075892,1.004624,0.883575,1.028561,1.178144e-16,251.296965,50.650454,59.879870
1,2021-01-07 02:00:00,30.844594,820.040961,0.642240,0.530100,4.160816,2.307342,627.485638,631.515731,630.323951,...,1.006158,0.943255,1.055169,0.985222,0.864358,1.011680,1.178144e-16,253.302595,51.009955,60.187294
2,2021-01-07 03:00:00,30.822927,820.059912,0.642791,0.530115,4.135495,2.294837,627.322589,631.205870,629.768853,...,1.010743,0.948550,1.060699,0.991934,0.871092,1.012111,1.178144e-16,254.730001,51.108173,60.164672
3,2021-01-07 04:00:00,30.859871,819.834424,0.641999,0.530127,4.200890,2.322727,627.760046,632.178559,630.788099,...,1.014600,0.950256,1.063285,0.994866,0.873887,1.010393,1.178144e-16,255.202556,51.248020,60.059837
4,2021-01-07 05:00:00,31.235552,819.353981,0.642314,0.530034,4.253487,2.296272,628.965906,632.865397,631.586442,...,1.026507,0.964205,1.078325,1.010437,0.885445,1.023722,1.178144e-16,253.684494,51.260781,59.829362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11300,2022-05-31 19:00:00,34.921922,817.960936,0.599597,0.745289,4.336915,2.114046,640.415246,644.650909,636.121706,...,1.158597,1.200987,1.345795,1.281682,1.220382,1.469659,4.271138e+01,233.667061,50.017260,53.136514
11301,2022-05-31 20:00:00,34.930163,818.299786,0.599503,0.747182,4.321065,2.336833,641.535940,645.780292,637.346623,...,1.161251,1.202199,1.344702,1.283642,1.224183,1.472635,4.265844e+01,233.670287,49.536603,52.846304
11302,2022-05-31 21:00:00,35.170096,817.913417,0.595354,0.743222,4.326123,2.300841,638.992327,643.641218,634.130478,...,1.162531,1.202464,1.342520,1.285216,1.222200,1.469588,4.261256e+01,233.371803,49.418948,53.049036
11303,2022-05-31 22:00:00,35.429867,818.189503,0.591048,0.748198,4.342594,2.229080,638.544698,643.134239,633.456320,...,1.167090,1.208050,1.352378,1.289805,1.227885,1.472233,4.270312e+01,233.243962,50.064544,52.951541


In [ ]:
df_tmt = pd.read_excel("")

In [ ]:
# computing the upper triangle correlation matrix
corr_mat_7 = df_other_features.corr().abs()
upper_tri_7 = corr_mat_7.where(np.triu(np.ones(corr_mat_7.shape), k=1).astype(np.bool))
upper_tri_7

In [ ]:
# plotting correlation heatmap for Feed Temperature
plt.figure(figsize=(20,20))
dataplot = sb.heatmap(upper_tri_7, cmap="Blues", annot=True)
  
# displaying heatmap
# plt.show()

# save the plot as .png

plt.savefig(heat_path +"/other_features_corr_heatmap.png")

In [ ]:
# finding index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper_tri_7.columns if any(upper_tri_7[column] > 0.80)]
print(); print(to_drop) 

In [ ]:
df_other_features

# Variation Inflation Factor

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

In [ ]:
# X = df_other_features[list(df_other_features.columns)]

# vif_info = pd.DataFrame()
# vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# vif_info['Column'] = X.columns
# vif_info.sort_values('VIF', ascending=False)

In [ ]:
X = df_other_features.copy()

# vif = pd.DataFrame()

# cols = list(X.columns)


scaler = StandardScaler()


features1 = X.copy()
scaler1 = scaler.fit(features)
features_arr1 = scaler1.transform(features1)
features_scaled1 = pd.DataFrame(features_arr1, columns=features1.columns)

In [ ]:
# def compute_vif(X,threshold=10.0):
#     try:
#         cols = list(X.columns)
#         vif = pd.DataFrame()
#         features = X.copy()
#         # n = len(features_columns)
     
#         while (len(cols)>0):
#             scaler = StandardScaler()
#             scaler = scaler.fit(features)
#             features_arr = scaler.transform(features)
#             features_scaled = pd.DataFrame(features_arr, columns=features.columns)

#             vif['vif'] = [variance_inflation_factor(features_arr,i) for i in range(features_arr.shape[1])]
# #             vif['features'] = pd.DataFrame(features_arr,columns=features_scaled).columns
#             vif['features'] = features.columns.tolist()

#             feature_with_vif_max = vif[vif.columns[-1]].loc[vif[vif.columns[0]] == vif[vif.columns[0]].max()].values[0]
#             print(f"{feature_with_vif_max}")
#             vif_max = vif[vif.columns[0]].max()
#             print(f"{vif_max}")
#             if(vif_max>threshold):
#                 cols.remove(feature_with_vif_max)
# #                 features = features[cols]

#             else:
#                 break
#             features_df = features[cols] 
#             features = features_df.copy()
        
#         selected_features_VIF = cols

#         return selected_features_VIF
#     except Exception as err:
#           print(str(err))

In [ ]:
# X = df_other_features.copy()
# a = compute_vif(X)

In [ ]:
# cols = list(X.columns)

In [ ]:
# cols.remove('Total_Naphtha_Feed')

In [ ]:
def compute_vif(features_arr,features_columns):
    try:
        vif = pd.DataFrame()
        vif['vif'] = [variance_inflation_factor(features_arr,i) for i in range(features_arr.shape[1])]
        vif['features'] = pd.DataFrame(features_arr,columns=features_columns).columns
        
        return vif
    except Exception as err:
        print(str(err))

In [ ]:
vif_df1 = compute_vif(features_arr1, features_scaled1.columns)
vif_df1

In [ ]:
features2 = features1.drop(labels=[vif_df1['features'].loc[vif_df1['vif'] == vif_df1['vif'].max()].values[0]], axis=1)


scaler2 = scaler.fit(features2)
features_arr2 = scaler.transform(features2)
features_scaled2 = pd.DataFrame(features_arr2, columns=features2.columns)


vif_df2 = compute_vif(features_arr2, features_scaled2.columns)
vif_df2

In [ ]:
features3 = features2.drop(labels=[vif_df2['features'].loc[vif_df2['vif'] == vif_df2['vif'].max()].values[0]], axis=1)


scaler3 = scaler.fit(features3)
features_arr3 = scaler.transform(features3)
features_scaled3 = pd.DataFrame(features_arr3, columns=features3.columns)


vif_df3 = compute_vif(features_arr3, features_scaled3.columns)
vif_df3

In [ ]:
features4 = features3.drop(labels=[vif_df3['features'].loc[vif_df3['vif'] == vif_df3['vif'].max()].values[0]], axis=1)


scaler4 = scaler.fit(features4)
features_arr4 = scaler.transform(features4)
features_scaled4 = pd.DataFrame(features_arr4, columns=features4.columns)


vif_df4 = compute_vif(features_arr4, features_scaled4.columns)
vif_df4

In [ ]:
features5 = features4.drop(labels=[vif_df4['features'].loc[vif_df4['vif'] == vif_df4['vif'].max()].values[0]], axis=1)


scaler5 = scaler.fit(features5)
features_arr5 = scaler.transform(features5)
features_scaled5 = pd.DataFrame(features_arr5, columns=features5.columns)


vif_df5 = compute_vif(features_arr5, features_scaled5.columns)
vif_df5

In [ ]:
features6 = features5.drop(labels=[vif_df5['features'].loc[vif_df5['vif'] == vif_df5['vif'].max()].values[0]], axis=1)


scaler6 = scaler.fit(features6)
features_arr6 = scaler.transform(features6)
features_scaled6 = pd.DataFrame(features_arr6, columns=features6.columns)


vif_df6 = compute_vif(features_arr6, features_scaled6.columns)
vif_df6

In [ ]:
vif_df6[vif_df6.columns[-1]]

In [23]:
df_other_features

,Date,Total_Naphtha_Feed,Avg_COT,Total_Ds_Ratio,Wall_Fuel_Gas_Flow,Ground_Fuel_Gas_Flow,O2,CrossOverTemp_Coil1,CrossOverTemp_Coil2,CrossOverTemp_Coil3,...,CIP1,CIP2,CIP3,CIP4,CIP5,CIP6,BFW Rate,DS Temp,Feed Temp1,Feed Temp2
0,2021-01-07 01:00:00,30.808213,820.259940,0.643152,0.530118,4.102766,2.353679,630.542551,633.943171,632.920609,...,1.024428,0.962958,1.075892,1.004624,0.883575,1.028561,1.178144e-16,251.296965,50.650454,59.879870
1,2021-01-07 02:00:00,30.844594,820.040961,0.642240,0.530100,4.160816,2.307342,627.485638,631.515731,630.323951,...,1.006158,0.943255,1.055169,0.985222,0.864358,1.011680,1.178144e-16,253.302595,51.009955,60.187294
2,2021-01-07 03:00:00,30.822927,820.059912,0.642791,0.530115,4.135495,2.294837,627.322589,631.205870,629.768853,...,1.010743,0.948550,1.060699,0.991934,0.871092,1.012111,1.178144e-16,254.730001,51.108173,60.164672
3,2021-01-07 04:00:00,30.859871,819.834424,0.641999,0.530127,4.200890,2.322727,627.760046,632.178559,630.788099,...,1.014600,0.950256,1.063285,0.994866,0.873887,1.010393,1.178144e-16,255.202556,51.248020,60.059837
4,2021-01-07 05:00:00,31.235552,819.353981,0.642314,0.530034,4.253487,2.296272,628.965906,632.865397,631.586442,...,1.026507,0.964205,1.078325,1.010437,0.885445,1.023722,1.178144e-16,253.684494,51.260781,59.829362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11300,2022-05-31 19:00:00,34.921922,817.960936,0.599597,0.745289,4.336915,2.114046,640.415246,644.650909,636.121706,...,1.158597,1.200987,1.345795,1.281682,1.220382,1.469659,4.271138e+01,233.667061,50.017260,53.136514
11301,2022-05-31 20:00:00,34.930163,818.299786,0.599503,0.747182,4.321065,2.336833,641.535940,645.780292,637.346623,...,1.161251,1.202199,1.344702,1.283642,1.224183,1.472635,4.265844e+01,233.670287,49.536603,52.846304
11302,2022-05-31 21:00:00,35.170096,817.913417,0.595354,0.743222,4.326123,2.300841,638.992327,643.641218,634.130478,...,1.162531,1.202464,1.342520,1.285216,1.222200,1.469588,4.261256e+01,233.371803,49.418948,53.049036
11303,2022-05-31 22:00:00,35.429867,818.189503,0.591048,0.748198,4.342594,2.229080,638.544698,643.134239,633.456320,...,1.167090,1.208050,1.352378,1.289805,1.227885,1.472233,4.270312e+01,233.243962,50.064544,52.951541


In [24]:
# read the imputed TMT data
df_tmt = pd.read_excel("BA_106_TMT_imputed_hourly_data.xlsx")

In [25]:
df_tmt

,Date,Max TMT,Coil 1,Coil 2,Coil 3,Coil 4,Coil 5,Coil 6
0,2021-01-07 01:00:00,995.0,989.0,988.0,983.0,995.0,989.0,988.0
1,2021-01-07 02:00:00,995.0,989.0,988.0,983.0,995.0,989.0,988.0
2,2021-01-07 03:00:00,995.0,989.0,988.0,983.0,995.0,989.0,988.0
3,2021-01-07 04:00:00,995.0,989.0,988.0,983.0,995.0,989.0,988.0
4,2021-01-07 05:00:00,995.0,989.0,988.0,983.0,995.0,989.0,988.0
...,...,...,...,...,...,...,...,...
11467,2022-06-01 19:00:00,1047.0,1020.0,1027.0,1002.0,1025.0,1010.0,1047.0
11468,2022-06-01 20:00:00,1047.0,1020.0,1027.0,1002.0,1025.0,1010.0,1047.0
11469,2022-06-01 21:00:00,1047.0,1020.0,1027.0,1002.0,1025.0,1010.0,1047.0
11470,2022-06-01 22:00:00,1047.0,1020.0,1027.0,1002.0,1025.0,1010.0,1047.0


In [26]:
# store Max TMT and date columns into a separate dataframe
df_max_tmt = df_tmt[['Date', 'Max TMT']]
df_max_tmt

,Date,Max TMT
0,2021-01-07 01:00:00,995.0
1,2021-01-07 02:00:00,995.0
2,2021-01-07 03:00:00,995.0
3,2021-01-07 04:00:00,995.0
4,2021-01-07 05:00:00,995.0
...,...,...
11467,2022-06-01 19:00:00,1047.0
11468,2022-06-01 20:00:00,1047.0
11469,2022-06-01 21:00:00,1047.0
11470,2022-06-01 22:00:00,1047.0


In [27]:
# merge df_max_tmt and df_other_features based on date column
df_merged = df_max_tmt.merge(df_other_features, on='Date')

In [29]:
os.getcwd()

'C:\\Users\\SABYA\\Aritar01\\CAP_106\\Raw_Data'

In [30]:
df_merged.to_excel("BA_106_merged_TMT_and_final_selected_features.xlsx", index=False)